In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [5]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [6]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [7]:
# https://www.tensorflow.org/install/source_windows

In [8]:
if os.name=='nt':
!pip install wget
import wget

In [9]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [10]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.6.1.3-2ubuntu5.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1697012 sha256=d5946384271d0fffae7df9239399afe86201c6888f46a31ec150507eb0aba584
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ge558l1/wheels/13/dc/b4/ca25040453ba296d853cb0ccf7c827a599f84e993647b41f42
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [12]:
import object_detection

In [13]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2023-04-10 12:31:44--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.79.128, 2a00:1450:4013:c05::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.79.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  --.-KB/s    in 0.1s    

2023-04-10 12:31:44 (145 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8

# 2. Create Label Map

In [14]:
labels = [{'name':'license', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [15]:
import shutil
if not os.path.exists('/content/Tensorflow/workspace/images/test'):
  shutil.copytree('/content/drive/MyDrive/License Plate/Tensorflow/workspace/images/test','/content/Tensorflow/workspace/images/test')
  shutil.copytree('/content/drive/MyDrive/License Plate/Tensorflow/workspace/images/train','/content/Tensorflow/workspace/images/train')

In [16]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [17]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 4. Copy Model Config to Training Folder

In [18]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning

In [19]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [20]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [21]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [22]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [23]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [24]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 8. Load Train Model From Checkpoint

In [25]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [26]:
# Prevent GPU complete consumption
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try: 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RunTimeError as e:
        print(e)

In [27]:
if not os.path.isdir('/content/drive/MyDrive/License Plate/Tensorflow/workspace/models/my_ssd_mobnet/ckpt-52.index'):
  shutil.copy('/content/drive/MyDrive/License Plate/Tensorflow/workspace/models/my_ssd_mobnet/ckpt-52.index', '/content/Tensorflow/workspace/models/my_ssd_mobnet')

if not os.path.isdir('/content/drive/MyDrive/License Plate/Tensorflow/workspace/models/my_ssd_mobnet/ckpt-52.data-00000-of-00001'):
  shutil.copy('/content/drive/MyDrive/License Plate/Tensorflow/workspace/models/my_ssd_mobnet/ckpt-52.data-00000-of-00001', '/content/Tensorflow/workspace/models/my_ssd_mobnet')

In [28]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-52')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect from an Image

In [29]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [30]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [31]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 KB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [32]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 782.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not curre

In [60]:
import easyocr
sub_list = []
sub_list2  = []
sub_list3 = []
sub_list4 = []

In [61]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [62]:
def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate


def ocr_it(image, detections, detection_threshold, region_threshold):
    
    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]
    
    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)
        
        text = filter_text(region, ocr_result, region_threshold)
        
        #plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
        #plt.show()
        #print(text)
    return text, region

In [63]:
IMAGE_PATH = os.path.join('/content/drive/MyDrive/License Plate/Test/imgs')

In [66]:
for x in range(901, 1001):
  name = str(x)+'.jpg'
  IMG_PATH = os.path.join(IMAGE_PATH, name)
  #print(IMG_PATH)
  if os.path.isfile(IMG_PATH):
    img = cv2.imread(IMG_PATH)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=2,
                min_score_thresh=.3,
                agnostic_mode=False)
    
    detection_threshold = 0.2
    region_threshold = 0.001
    image = image_np_with_detections

    try:
      text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)
    except UnboundLocalError:
      text='0000000'

    if  len(text) == 0:
      text.append('0000000')
    
    if len(text) ==1:
      text =text[0].split( )
      first = text[0]
      if 'W' in first:
        first = first.replace('W', '11')
      second = text[-1]
    elif len(text) == 2:
      first = text[0].split( )[0]
      if 'W' in first:
        first = first.replace('W', '11')
      second = text[-1]

    new_first = ''
    new_second = ''
    for n in first:
      if n.isnumeric():
        new_first+=n
    for n in second:
      if n.isnumeric():
        new_second+=n
    first=new_first
    second=new_second

    lis = []
    if len(second) < 4:
      second = '0'+second
    elif len(second) == 4:
      if len(first) > 3:
        first = '0'+first 
    if len(first) < 3:
      first = '0'+first

    print(first)
    print(second)

    try:
      lis.append(int(first[0]))
    except:
      lis.append(0)
    try:
      lis.append(int(first[1]))
    except:
      lis.append(0)
    try:
      lis.append(int(first[2]))
    except:
      lis.append(0)
      
    
    try:
      lis.append(int(second[-4]))
    except:
      lis.append(0)
    try:
      lis.append(int(second[-3]))
    except:
      lis.append(0)
    try:
      lis.append(int(second[-2]))
    except:
      lis.append(0)
    try:
      lis.append(int(second[-1]))
    except:
      lis.append(0)

    sub_list.append(lis)
  

#plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
#plt.show()

1730859116
1730859116
071
1526
092
3039
076
0271
1915583
1915583
180
0228
0
6070
004
9416
24591398
24591398
1276274029
1276274029
1276274029
1276274029
09
09
703540628
703540628
49651332
49651332
095
01
01436
4790
1823256149
1823256149
088
6386
158
54820
158
54820
105
9699
151
80951
109358
109358
050
0
152
0938
016
3660
753358959
753358959
05390
5390
145
1093
156
0864
01813
9656
0000000
0000000
673
55998
7190832123
7190832123
7163668
7163668
087
7369
062
9640
160844041
160844041
140
0659
096
3734
062
6569
34354413
34354413
135
5906
135
5906
6259734
6259734
01
3691
015
9677
015
9677
015
9677
015
9677
091
4293
159858894
159858894
090
9848
122
2171
076
392714
167395257
167395257
135
5906
01405
9240
167
2142
00
009
0
0
119
6926
01336950
01336950
272
59295
272
59295
01
01
0017754
8553
14490651
14490651
14490651
14490651
12854
0177
079
9778
0125033
6088
094
0573
661
2580
195
51823
051231866
051231866
108
6543
0121535
6559
096
5370
152
3567
17856291
17856291
1352788
1352788
008
52903
0111353


In [65]:
len(sub_list)

98

In [67]:
len(sub_list2)

112

In [ ]:
916,1097

In [68]:
sub_listN = sub_list+sub_list2

In [69]:
len(sub_listN)

210

In [70]:
from tensorflow.keras.utils import to_categorical as tcl
import pandas as pd

In [71]:
df = pd.read_csv(r'/content/drive/MyDrive/License Plate/Test/SampleSubmission (1).csv')
id = df["id"].to_list()
id

['img_901_1',
 'img_901_2',
 'img_901_3',
 'img_901_4',
 'img_901_5',
 'img_901_6',
 'img_901_7',
 'img_902_1',
 'img_902_2',
 'img_902_3',
 'img_902_4',
 'img_902_5',
 'img_902_6',
 'img_902_7',
 'img_903_1',
 'img_903_2',
 'img_903_3',
 'img_903_4',
 'img_903_5',
 'img_903_6',
 'img_903_7',
 'img_904_1',
 'img_904_2',
 'img_904_3',
 'img_904_4',
 'img_904_5',
 'img_904_6',
 'img_904_7',
 'img_905_1',
 'img_905_2',
 'img_905_3',
 'img_905_4',
 'img_905_5',
 'img_905_6',
 'img_905_7',
 'img_906_1',
 'img_906_2',
 'img_906_3',
 'img_906_4',
 'img_906_5',
 'img_906_6',
 'img_906_7',
 'img_907_1',
 'img_907_2',
 'img_907_3',
 'img_907_4',
 'img_907_5',
 'img_907_6',
 'img_907_7',
 'img_908_1',
 'img_908_2',
 'img_908_3',
 'img_908_4',
 'img_908_5',
 'img_908_6',
 'img_908_7',
 'img_909_1',
 'img_909_2',
 'img_909_3',
 'img_909_4',
 'img_909_5',
 'img_909_6',
 'img_909_7',
 'img_910_1',
 'img_910_2',
 'img_910_3',
 'img_910_4',
 'img_910_5',
 'img_910_6',
 'img_910_7',
 'img_911_1',
 'img_

In [78]:
new_list_sub = []
data = tcl(sub_listN, 10, dtype='int32')
for mm in data:
    for xx in mm:
      new_list_sub.append(list(xx))
new_list_sub

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0,

In [79]:
liss = list(zip(id, new_list_sub))
liss = pd.DataFrame(liss, columns = ['img', '0'])
#liss[['0','1', '2', '3', '4', '5']] = pd.DataFrame(df2.'0'.tolist(), index= df2.index)
print(liss)


             img                               0
0      img_901_1  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
1      img_901_2  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
2      img_901_3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
3      img_901_4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
4      img_901_5  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
...          ...                             ...
1465  img_1113_3  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1466  img_1113_4  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
1467  img_1113_5  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
1468  img_1113_6  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
1469  img_1113_7  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

[1470 rows x 2 columns]


In [80]:
df3 = pd.DataFrame(liss['0'].to_list(), columns=['0','1', '2', '3', '4', '5', '6', '7', '8', '9'])
df3.insert(0, "img", id, True)
print(df3)

             img  0  1  2  3  4  5  6  7  8  9
0      img_901_1  0  1  0  0  0  0  0  0  0  0
1      img_901_2  0  0  0  0  0  0  1  0  0  0
2      img_901_3  0  0  0  0  0  0  0  0  0  1
3      img_901_4  0  0  0  0  0  0  0  0  0  1
4      img_901_5  0  0  0  0  0  0  1  0  0  0
...          ... .. .. .. .. .. .. .. .. .. ..
1465  img_1113_3  1  0  0  0  0  0  0  0  0  0
1466  img_1113_4  0  0  0  0  0  1  0  0  0  0
1467  img_1113_5  0  0  0  1  0  0  0  0  0  0
1468  img_1113_6  0  0  0  1  0  0  0  0  0  0
1469  img_1113_7  0  0  0  1  0  0  0  0  0  0

[1470 rows x 11 columns]


In [81]:
df3.to_csv('/content/drive/MyDrive/License_Plate_submit.csv', index=False)